In [80]:
import pandas as pd

In [81]:
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

In [83]:
import sqlite3

In [82]:
# Database connection
conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()

In [59]:
# Creating tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INTEGER PRIMARY KEY,
        Name TEXT,
        Type TEXT,
        Address TEXT,
        City TEXT,
        Contact TEXT
    )
''')

In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID INTEGER PRIMARY KEY,
        Name TEXT,
        Type TEXT,
        City TEXT,
        Contact TEXT
    )
''')

In [8]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS food_listings (
        Food_ID INTEGER PRIMARY KEY,
        Food_Name TEXT,
        Quantity TEXT,
        Expiry_Date TEXT,
        Provider_ID INTEGER,
        Provider_Type TEXT,      
        Location TEXT,
        Food_Type TEXT,
        Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
    )
''')

In [9]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID INTEGER PRIMARY KEY,
        Food_ID INTEGER,
        Receiver_ID INTEGER,
        Status TEXT,
        TimeStamp TEXT,
        FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
        FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
        
    )
''')

In [10]:
for index, row in providers.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (?, ?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()


In [11]:
for _, row in receivers.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO receivers (Receiver_ID, Name, Type, City, Contact)
        VALUES (?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()



In [12]:
for _, row in food_listings.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO food_listings 
        (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()


In [30]:
for _, row in claims.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO claims 
        (Claim_ID, Food_ID, Receiver_ID, Status, TimeStamp)
        VALUES (?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()

In [33]:
# SQL query to get the city with the highest number of providers
query = """
    SELECT City, COUNT(*) AS Provider_Count
    FROM providers
    GROUP BY City
    ORDER BY Provider_Count DESC
    LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "Provider_Count"])
df

,City,Provider_Count
0,South Christopherborough,3


In [ ]:
#1 How many food providers and receivers are there in each city?

query = """
    SELECT
    COALESCE(p.City, r.City) AS City,
    COUNT(DISTINCT p.provider_id) AS provider_count,
    COUNT(DISTINCT r.receiver_id) AS receiver_count
FROM providers p
FULL OUTER JOIN receivers r
    ON p.City = r.City
GROUP BY COALESCE(p.City, r.City)
ORDER BY City;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "provider_count", "receiver_count"])


In [ ]:
#2

query = """
    SELECT 
    provider_type,
    SUM(quantity) AS total_quantity
FROM food_listings
GROUP BY provider_type order by total_quantity DESC LIMIT 1;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Provider_Type", "total_quantity"])
df

,Provider_Type,total_quantity
0,Restaurant,6923


In [ ]:
#3

query = """
   SELECT City, Contact from providers where City == "New Jessica"

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Provider_Type", "total_quantity"])
df

,Provider_Type,total_quantity
0,New Jessica,+1-600-220-0480


In [ ]:
#4

query = """
   WITH claim_counts AS (
    SELECT 
        r.Receiver_ID,
        r.Name,
        COUNT(c.Claim_ID) AS claim_count
    FROM 
        receivers r
    LEFT JOIN 
        claims c
    ON 
        r.Receiver_ID = c.Receiver_ID
    GROUP BY 
        r.Receiver_ID, r.Name
)
SELECT *
FROM claim_counts
WHERE claim_count = (
    SELECT MAX(claim_count) FROM claim_counts
);


"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Reciever_ID","Name", "claim_count"])
df

,Reciever_ID,Name,claim_count
0,276,Scott Hunter,5
1,371,William Frederick,5
2,742,Matthew Webb,5
3,800,Anthony Garcia,5


In [ ]:
#5

query = """
   SELECT 
    f.provider_id,
    p.Name,
    SUM(f.quantity) AS total_quantity
FROM 
    food_listings f
JOIN 
    providers p
ON 
    f.provider_id = p.provider_id
GROUP BY 
    f.provider_id, p.Name;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Provider_ID","Name", "total_quantity"])


In [ ]:
#6

query = """
WITH location_counts AS (
    SELECT 
        Location as city,
        COUNT(*) AS listing_count
    FROM 
        food_listings
    GROUP BY 
        Location
)
SELECT *
FROM location_counts
WHERE listing_count = (
    SELECT MAX(listing_count) 
    FROM location_counts
);



"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["city","listing_count"])
df


,city,listing_count
0,New Carol,6
1,South Kathryn,6


In [97]:
#7

query = """
SELECT 
    Food_Name,
    COUNT(*) AS listing_count,
    SUM(Quantity) AS total_quantity
FROM 
    food_listings
GROUP BY 
    Food_Name
ORDER BY 
    listing_count DESC;


"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Food_Name","listing_count", "total_quantity"])
df


,Food_Name,listing_count,total_quantity
0,Rice,114,3133
1,Soup,111,2819
2,Salad,105,2765
3,Dairy,103,2765
4,Chicken,103,2324
5,Pasta,102,2626
6,Bread,98,2726
7,Fish,92,2081
8,Vegetables,91,2342
9,Fruits,81,2213


In [98]:
#8 

query = """
SELECT 
    f.Food_Name,
    COUNT(*) AS claim_count
FROM 
    claims c
JOIN 
    food_listings f
ON 
    c.Food_ID = f.Food_ID
GROUP BY 
    f.Food_Name;


"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Food_Name","claim_count"])
df


,Food_Name,claim_count
0,Bread,94
1,Chicken,102
2,Dairy,110
3,Fish,108
4,Fruits,71
5,Pasta,87
6,Rice,122
7,Salad,106
8,Soup,114
9,Vegetables,86


In [100]:
#9

query = """
WITH completed_counts AS (
    SELECT 
        f.Provider_ID,
        COUNT(*) AS completed_claims
    FROM 
        food_listings f
    JOIN 
        claims c
        ON f.Food_ID = c.Food_ID
    WHERE 
        c.Status = 'Completed'
    GROUP BY 
        f.Provider_ID
)
SELECT 
    p.Provider_ID,
    p.Name,
    cc.completed_claims
FROM 
    completed_counts cc
JOIN 
    providers p
    ON cc.Provider_ID = p.Provider_ID
WHERE 
    cc.completed_claims = (
        SELECT MAX(completed_claims) FROM completed_counts
    );


"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Food_ID","Name","completed_count"])
df


,Food_ID,Name,completed_count
0,709,Barry Group,5


In [101]:
#10

query = """
SELECT 
    Status,
    COUNT(*) AS claim_count,
    ROUND( (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims)), 2 ) AS percentage
FROM 
    claims
GROUP BY 
    Status;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Status","claim_count","percentage"])
df


,Status,claim_count,percentage
0,Cancelled,336,33.6
1,Completed,339,33.9
2,Pending,325,32.5


In [104]:
#10

query = """
SELECT 
    Status,
    COUNT(*) AS claim_count,
    ROUND( (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims)), 2 ) AS percentage
FROM 
    claims
GROUP BY 
    Status;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Status","claim_count","percentage"])
df


,Status,claim_count,percentage
0,Cancelled,336,33.6
1,Completed,339,33.9
2,Pending,325,32.5


In [ ]:
#11

query = """
SELECT 
    r.Receiver_ID,
    r.Name,
    ROUND(AVG(f.Quantity), 2) AS avg_claimed_quantity
FROM 
    claims c
JOIN 
    food_listings f
    ON c.Food_ID = f.Food_ID
JOIN 
    receivers r
    ON c.Receiver_ID = r.Receiver_ID
GROUP BY 
    r.Receiver_ID, r.Name;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Receiver_ID","Name","avg_claimed_quantity"])
df


In [107]:
#12

query = """
SELECT 
    f.Meal_Type,
    COUNT(c.Claim_ID) AS claim_count
FROM 
    claims c
JOIN 
    food_listings f
    ON c.Food_ID = f.Food_ID
GROUP BY 
    f.Meal_Type
ORDER BY 
    claim_count DESC
LIMIT 1;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Meal_Type","claim_count"])
df


,Meal_Type,claim_count
0,Breakfast,278


In [108]:
#13

query = """
SELECT 
    f.Provider_ID,
    p.Name,
    SUM(f.Quantity) AS total_quantity
FROM 
    food_listings f
JOIN 
    providers p
    ON f.Provider_ID = p.Provider_ID
GROUP BY 
    f.Provider_ID, p.Name
ORDER BY 
    p.Provider_ID;

"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["Provider_ID","Name","total_quantity"])
df


,Provider_ID,Name,total_quantity
0,1,Gonzales-Cochran,78
1,2,"Nielsen, Johnson and Fuller",53
2,4,"Clark, Prince and Williams",96
3,5,Coleman-Farley,92
4,7,Ortiz-Lee,114
5,9,Nguyen Inc,21
6,10,"Hall, Nguyen and Martinez",39
7,11,"Flores, Jackson and Ford",27
8,14,Nguyen-Tran,23
9,16,Brown and Sons,47
